In [ ]:
#!pip install -q torch==1.7.1+cpu torchvision==0.8.2+cpu torchaudio==0.7.2 torchtext==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
#!pip install -q --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
#!pip install -q --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
#!pip install -q --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
#!pip install -q --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
#!pip install torch-geometric

In [1]:
import torch
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from sklearn.neural_network import MLPClassifier 
from sklearn.linear_model import LogisticRegressionCV

from sklearn.ensemble import GradientBoostingClassifier
from torch_geometric.data import NeighborSampler
from torch.optim import lr_scheduler
import torch.optim as optim
import matplotlib.pyplot as plt
import collections
import numpy as np
from modules.model import Net
from modules.sampling import Sampler

In [2]:
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = data.x.to(device)
y = data.y.squeeze().to(device)

In [3]:
train_mask = torch.tensor([True]*int(0.8*len(data.x)+1) + [False]*int(0.2*len(data.x)))
#val_mask = torch.tensor([False]*int(0.6*len(data.x)+1) + [True]*int(0.2*len(data.x)+1)+[False]*int(0.2*len(data.x)))
test_mask = torch.tensor([False]*int(0.8*len(data.x)+1) + [True]*int(0.2*len(data.x)))

In [4]:
#train_mask = data.train_mask
#val_mask = data.val_mask
#test_mask = data.test_mask

In [5]:
from sklearn.metrics import precision_score
def train(model,data,optimizer,Sampler,train_loader,dropout):
    model.train()        
    total_loss = 0
    optimizer.zero_grad()
    if model.mode == 'unsupervised':
        if model.conv=='GCN':
            arr=torch.nonzero(train_mask == True)
            indices_of_train_data = ([item for sublist in arr for item in sublist])
            out = model.inference(data.to(device),dp=dropout)
            samples = Sampler.sample(indices_of_train_data)
            loss = model.loss(out[train_mask], samples)
            total_loss+=loss
        else:
            for batch_size, n_id, adjs in train_loader:
                # adjs holds a list of (edge_index, e_id, size) tuples.
                adjs = [adj for adj in adjs]
                out = model.forward(data.x[n_id].to(device), adjs)
                samples = Sampler.sample((n_id.numpy().tolist())[:batch_size])
                loss = model.loss(out, samples)#pos_batch.to(device), neg_batch.to(device))
                #print(out.shape, samples[0].shape,samples[1].shape)
                total_loss+=loss
        total_loss.backward()
        optimizer.step()      
        return total_loss /len(train_loader)
    elif model.mode== 'supervised':
        if model.conv=='GCN':
            out = model.inference(data.to(device),dp=dropout)
            loss = model.loss_sup(out[train_mask],y[train_mask])
            total_loss+=loss
        else:
            for batch_size, n_id, adjs in train_loader:
                adjs = [adj for adj in adjs]
                out = model.forward(data.x[n_id].to(device), adjs)
                loss = model.loss_sup(out,y[n_id[:batch_size]])
                total_loss+=loss
        total_loss.backward(retain_graph=True)
        optimizer.step()      
        return total_loss /len(train_loader)       

@torch.no_grad()
def test(model,data,classifier): 
    model.eval()
    out = model.inference(data.to(device))
    y_true = y.cpu().detach().numpy()
    if model.mode == 'supervised':
        y_true = y.cpu().unsqueeze(-1)
        y_pred = out.cpu().argmax(dim=-1, keepdim=True)
        accs = []
        for mask in [train_mask, val_mask, test_mask]:    
            accs+=[int(y_pred[mask].eq(y_true[mask]).sum()) / int(mask.sum())]
        return accs
    elif model.mode == 'unsupervised': 
        if classifier == 'logistic regression':
            clf = LogisticRegressionCV(cv = 5, max_iter = 3000).fit(out.cpu().detach()[train_mask].numpy(), y.cpu().detach()[train_mask].numpy())
        else:
            clf = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1, max_depth=3, random_state=0).fit(out.cpu().detach()[train_mask].numpy(), y.cpu().detach()[train_mask].numpy())
        #clf = MLPClassifier(random_state=1, max_iter=3000).fit(out.cpu().detach()[train_mask].numpy(), y.cpu().detach()[train_mask].numpy())
        accs = []
        for mask in [train_mask, test_mask]:    
            accs += [(clf.score(out.cpu().detach()[mask].numpy(), y.cpu().detach()[mask].cpu().numpy()))]
        return accs
        
        #return [precision_score(y_true[train_mask.cpu()], best_preds_train, average='macro'), precision_score(y_true[val_mask.cpu()], best_preds_val, average='macro'),precision_score(y_true[test_mask.cpu()], best_preds, average='macro')]
    

**Для начала посмотрим, что получится, если передать только фичи. Без обучения**

**Теперь сравним с результатом после обучения эмбедингов**

In [ ]:
SAGE = {"Name":"SAGE" , "walk length":5,"walks per node":50,"num negative samples":20,"context size" : 10,"p":1,"q":1, "loss var": "Random Walks"}
DeepWalk = {"Name": "DeepWalk","walk length":10,"walks per node":10,"num negative samples":10,"context size" : 10,"p":1,"q":1,"loss var": "Random Walks" } #Проблемы с памятью после того, как увеличила количество тренировочных данных
#DeepWalk = {"Name": "DeepWalk","walk length":40,"walks per node":80,"num negative samples":20,"context size" : 10,"p":1,"q":1,"loss var": "Random Walks" } #Проблемы с памятью после того, как увеличила количество тренировочных данных
Node2Vec = {"Name": "Node2Vec","walk length":20,"walks per node":10,"num negative samples":1,"context size" : 10,"p":1.414 ,"q":1.414, "loss var": "Random Walks"}#то же самое 
#Node2Vec = {"Name": "Node2Vec","walk length":100,"walks per node":18,"num negative samples":20,"context size" : 16,"p":1.414 ,"q":1.414, "loss var": "Random Walks"}#то же самое 

LINE2 = {"Name": "LINE2","C": "Adj","num negative samples":10,"loss var": "Context Matrix"} #настроить параметры в принципе при 0.003 что то выходит  
HOPE_RPR = {"Name": "HOPE: RPR","C":"RPR","loss var": "Factorization"}#оч большой loss


GraphFactorization = {"Name": "Graph Factorization","C":"Adj","loss var": "Factorization"} #jоч большой loss
HOPE_Katz = {"Name": "HOPE: Katz","C":"Katz","loss var": "Factorization"}#Проблемы оч большой loss
VERSE_PPR =  {"Name": "VERSE","C": "PPR","num negative samples":10,"loss var": "Context Matrix"}  #проблеиы


**SAGE [5]**

**[25]**

In [10]:
import optuna
from torch_geometric.datasets import Planetoid
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]

def objective(trial):
    # Integer parameter
    hidden_layer = trial.suggest_categorical("hidden_layer", [32,64,128,256])
    out_layer = trial.suggest_categorical("out_layer", [32,64,128])
    # Floating point parameter
    dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
    size = [5,10,25]#list(map(lambda x: int(x),b))
    Conv ="SAGE"# trial.suggest_categorical("conv", ["SAGE","GCN","GAT"])
    
    walk_length = trial.suggest_int("walk length",5,20,step = 5)
    walks_per_node = trial.suggest_int("walks per node", 5,10,step = 5)
    context_size = trial.suggest_int("context size",5,20,step = 5)
    q = trial.suggest_float("q", 0.0,0.9,step = 0.1)
    p = trial.suggest_float("p", 0.0,0.9,step = 0.1)
    num_negative_samples = walks_per_node
    loss = {"Name": "Node2Vec","walk length":walk_length,"walks per node":walks_per_node,"num negative samples":num_negative_samples,"context size" : context_size ,"p":p ,"q":q, "loss var": "Random Walks"}#то же самое 

    model = Net(dataset = dataset,mode='unsupervised',conv=Conv,loss_function=loss["loss var"],device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = len(size),dropout = dropout)
    train_loader = NeighborSampler(data.edge_index, node_idx=train_mask, batch_size = 677, sizes=size)
    LossSampler = Sampler(data,device=device,mask=train_mask,loss_info=loss)
    
    model.to(device)
    #Generate the optimizers
#optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop","SGD"])
    learning_rate=trial.suggest_float("lr",5e-4,1e-2)
    #optimizer = getattr(optim,optimizer_name)(model.parameters(),lr = learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  
    #classifier = trial.suggest_categorical("classifier", ["logistic regression", "catboost"])
    classifier = "logistic regression"
   # classifier = "catboost"
    #training of the model
    for epoch in range(50):
        loss = train(model,data,optimizer,LossSampler,train_loader,dropout)
        train_acc, test_acc = test(model,data,classifier)
        
    trial.report(test_acc,epoch)
    if trial.should_prune():
        raise optuna.exceptions.TrialsPruned()
    return test_acc
        
    
study = optuna.create_study(direction="maximize",study_name="SAGE loss")
study.optimize(objective,n_trials = 1)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print(" Number of finid=shed trials: ",len(study.trials))
print(" Number of pruned trials: ", len(pruned_trials))
print(" Number of complete trials: ", len(complete_trials))

print('Best trial:')
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key,value))
    

[I 2021-04-11 21:51:45,759] A new study created in memory with name: SAGE loss
[I 2021-04-11 22:05:05,347] Trial 0 finished with value: 0.6580406654343808 and parameters: {'hidden_layer': 32, 'out_layer': 32, 'dropout': 0.4, 'walk length': 10, 'walks per node': 5, 'context size': 15, 'q': 0.4, 'p': 0.6000000000000001, 'lr': 0.005455707196236377}. Best is trial 0 with value: 0.6580406654343808.


Study statistics: 
 Number of finid=shed trials:  1
 Number of pruned trials:  0
 Number of complete trials:  1
Best trial:
 Value:  0.6580406654343808
 Params: 
 hidden_layer: 32
 out_layer: 32
 dropout: 0.4
 walk length: 10
 walks per node: 5
 context size: 15
 q: 0.4
 p: 0.6000000000000001
 lr: 0.005455707196236377


**[5,10]**

In [11]:
import optuna
from torch_geometric.datasets import Planetoid
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]

def objective(trial):
    # Integer parameter
    hidden_layer = trial.suggest_categorical("hidden_layer", [32,64,128,256])
    out_layer = trial.suggest_categorical("out_layer", [32,64,128])
    # Floating point parameter
    dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
    size = [5,10,25]#list(map(lambda x: int(x),b))
    Conv ="SAGE"# trial.suggest_categorical("conv", ["SAGE","GCN","GAT"])
    
    walk_length = trial.suggest_int("walk length",5,20,step = 5)
    walks_per_node = trial.suggest_int("walks per node", 5,10,step = 5)
    context_size = trial.suggest_int("context size",5,20,step = 5)
    q = trial.suggest_float("q", 0.0,0.9,step = 0.1)
    p = trial.suggest_float("p", 0.0,0.9,step = 0.1)
    num_negative_samples = walks_per_node
    loss = {"Name": "Node2Vec","walk length":walk_length,"walks per node":walks_per_node,"num negative samples":num_negative_samples,"context size" : context_size ,"p":p ,"q":q, "loss var": "Random Walks"}#то же самое 

    model = Net(dataset = dataset,mode='unsupervised',conv=Conv,loss_function=loss["loss var"],device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = len(size),dropout = dropout)
    train_loader = NeighborSampler(data.edge_index, node_idx=train_mask, batch_size = 677, sizes=size)
    LossSampler = Sampler(data,device=device,mask=train_mask,loss_info=loss)
    
    model.to(device)
    #Generate the optimizers
#optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop","SGD"])
    learning_rate=trial.suggest_float("lr",5e-4,1e-2)
    #optimizer = getattr(optim,optimizer_name)(model.parameters(),lr = learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  
    #classifier = trial.suggest_categorical("classifier", ["logistic regression", "catboost"])
    #classifier = "logistic regression"
    classifier = "catboost"
    #training of the model
    for epoch in range(50):
        loss = train(model,data,optimizer,LossSampler,train_loader,dropout)
        train_acc, test_acc = test(model,data,classifier)
        
    trial.report(test_acc,epoch)
    if trial.should_prune():
        raise optuna.exceptions.TrialsPruned()
    return test_acc
        
    
study = optuna.create_study(direction="maximize",study_name="SAGE loss")
study.optimize(objective,n_trials = 1)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print(" Number of finid=shed trials: ",len(study.trials))
print(" Number of pruned trials: ", len(pruned_trials))
print(" Number of complete trials: ", len(complete_trials))

print('Best trial:')
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key,value))

[I 2021-04-11 22:05:14,267] A new study created in memory with name: SAGE loss
[I 2021-04-11 22:16:10,627] Trial 0 finished with value: 0.6857670979667283 and parameters: {'hidden_layer': 128, 'out_layer': 128, 'dropout': 0.5, 'walk length': 20, 'walks per node': 10, 'context size': 10, 'q': 0.8, 'p': 0.2, 'lr': 0.009088122312325303}. Best is trial 0 with value: 0.6857670979667283.


Study statistics: 
 Number of finid=shed trials:  1
 Number of pruned trials:  0
 Number of complete trials:  1
Best trial:
 Value:  0.6857670979667283
 Params: 
 hidden_layer: 128
 out_layer: 128
 dropout: 0.5
 walk length: 20
 walks per node: 10
 context size: 10
 q: 0.8
 p: 0.2
 lr: 0.009088122312325303


In [ ]:
import optuna
from torch_geometric.datasets import Planetoid
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]

def objective(trial):
    # Integer parameter
    hidden_layer = trial.suggest_categorical("hidden_layer", [32,64,128,256])
    out_layer = trial.suggest_categorical("out_layer", [32,64,128])
    # Floating point parameter
    dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
    size = [5,10]#list(map(lambda x: int(x),b))
    Conv ="GCN"# trial.suggest_categorical("conv", ["SAGE","GCN","GAT"])
    
    walk_length = trial.suggest_int("walk length",5,20,step = 5)
    walks_per_node = trial.suggest_int("walks per node", 5,10,step = 5)
    context_size = trial.suggest_int("context size",5,20,step = 5)
    q = trial.suggest_float("q", 0.0,0.9,step = 0.1)
    p = trial.suggest_float("p", 0.0,0.9,step = 0.1)
    num_negative_samples = walks_per_node
    loss = {"Name": "Node2Vec","walk length":walk_length,"walks per node":walks_per_node,"num negative samples":num_negative_samples,"context size" : context_size ,"p":p ,"q":q, "loss var": "Random Walks"}#то же самое 

    model = Net(dataset = dataset,mode='unsupervised',conv=Conv,loss_function=loss["loss var"],device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = len(size),dropout = dropout)
    train_loader = NeighborSampler(data.edge_index, node_idx=train_mask, batch_size = 677, sizes=size)
    LossSampler = Sampler(data,device=device,mask=train_mask,loss_info=loss)
    
    model.to(device)
    #Generate the optimizers
#optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop","SGD"])
    learning_rate=trial.suggest_float("lr",5e-4,1e-2)
    #optimizer = getattr(optim,optimizer_name)(model.parameters(),lr = learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  
    #classifier = trial.suggest_categorical("classifier", ["logistic regression", "catboost"])
    #classifier = "logistic regression"
    classifier = "catboost"
    #training of the model
    for epoch in range(50):
        loss = train(model,data,optimizer,LossSampler,train_loader,dropout)
        train_acc, test_acc = test(model,data,classifier)
        
    trial.report(test_acc,epoch)
    if trial.should_prune():
        raise optuna.exceptions.TrialsPruned()
    return test_acc
        
    
study = optuna.create_study(direction="maximize",study_name="SAGE loss")
study.optimize(objective,n_trials = 1)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print(" Number of finid=shed trials: ",len(study.trials))
print(" Number of pruned trials: ", len(pruned_trials))
print(" Number of complete trials: ", len(complete_trials))

print('Best trial:')
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key,value))

**[10,25]**

In [ ]:
import optuna
from torch_geometric.datasets import Planetoid
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]

def objective(trial):
    # Integer parameter
    hidden_layer = trial.suggest_categorical("hidden_layer", [32,64,128,256])
    out_layer = trial.suggest_categorical("out_layer", [32,64,128])
    # Floating point parameter
    dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
    size = [10,25]#list(map(lambda x: int(x),b))
    Conv ="GCN"# trial.suggest_categorical("conv", ["SAGE","GCN","GAT"])
    
    walk_length = trial.suggest_int("walk length",5,20,step = 5)
    walks_per_node = trial.suggest_int("walks per node", 5,10,step = 5)
    context_size = trial.suggest_int("context size",5,20,step = 5)
    q = trial.suggest_float("q", 0.0,0.9,step = 0.1)
    p = trial.suggest_float("p", 0.0,0.9,step = 0.1)
    num_negative_samples = walks_per_node
    loss = {"Name": "Node2Vec","walk length":walk_length,"walks per node":walks_per_node,"num negative samples":num_negative_samples,"context size" : context_size ,"p":p ,"q":q, "loss var": "Random Walks"}#то же самое 

    model = Net(dataset = dataset,mode='unsupervised',conv=Conv,loss_function=loss["loss var"],device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = len(size),dropout = dropout)
    train_loader = NeighborSampler(data.edge_index, node_idx=train_mask, batch_size = 677, sizes=size)
    LossSampler = Sampler(data,device=device,mask=train_mask,loss_info=loss)
    
    model.to(device)
    #Generate the optimizers
#optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop","SGD"])
    learning_rate=trial.suggest_float("lr",5e-4,1e-2)
    #optimizer = getattr(optim,optimizer_name)(model.parameters(),lr = learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  
    #classifier = trial.suggest_categorical("classifier", ["logistic regression", "catboost"])
    classifier = "logistic regression"
    #classifier = "catboost"
    #training of the model
    for epoch in range(50):
        loss = train(model,data,optimizer,LossSampler,train_loader,dropout)
        train_acc, test_acc = test(model,data,classifier)
        
    trial.report(test_acc,epoch)
    if trial.should_prune():
        raise optuna.exceptions.TrialsPruned()
    return test_acc
        
    
study = optuna.create_study(direction="maximize",study_name="SAGE loss")
study.optimize(objective,n_trials = 1)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print(" Number of finid=shed trials: ",len(study.trials))
print(" Number of pruned trials: ", len(pruned_trials))
print(" Number of complete trials: ", len(complete_trials))

print('Best trial:')
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key,value))

In [ ]:
import optuna
from torch_geometric.datasets import Planetoid
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]

def objective(trial):
    # Integer parameter
    hidden_layer = trial.suggest_categorical("hidden_layer", [32,64,128,256])
    out_layer = trial.suggest_categorical("out_layer", [32,64,128])
    # Floating point parameter
    dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
    size = [10,25]#list(map(lambda x: int(x),b))
    Conv ="GCN"# trial.suggest_categorical("conv", ["SAGE","GCN","GAT"])
    
    walk_length = trial.suggest_int("walk length",5,20,step = 5)
    walks_per_node = trial.suggest_int("walks per node", 5,10,step = 5)
    context_size = trial.suggest_int("context size",5,20,step = 5)
    q = trial.suggest_float("q", 0.0,0.9,step = 0.1)
    p = trial.suggest_float("p", 0.0,0.9,step = 0.1)
    num_negative_samples = walks_per_node
    loss = {"Name": "Node2Vec","walk length":walk_length,"walks per node":walks_per_node,"num negative samples":num_negative_samples,"context size" : context_size ,"p":p ,"q":q, "loss var": "Random Walks"}#то же самое 

    model = Net(dataset = dataset,mode='unsupervised',conv=Conv,loss_function=loss["loss var"],device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = len(size),dropout = dropout)
    train_loader = NeighborSampler(data.edge_index, node_idx=train_mask, batch_size = 677, sizes=size)
    LossSampler = Sampler(data,device=device,mask=train_mask,loss_info=loss)
    
    model.to(device)
    #Generate the optimizers
#optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop","SGD"])
    learning_rate=trial.suggest_float("lr",5e-4,1e-2)
    #optimizer = getattr(optim,optimizer_name)(model.parameters(),lr = learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  
    #classifier = trial.suggest_categorical("classifier", ["logistic regression", "catboost"])
    #classifier = "logistic regression"
    classifier = "catboost"
    #training of the model
    for epoch in range(50):
        loss = train(model,data,optimizer,LossSampler,train_loader,dropout)
        train_acc, test_acc = test(model,data,classifier)
        
    trial.report(test_acc,epoch)
    if trial.should_prune():
        raise optuna.exceptions.TrialsPruned()
    return test_acc
        
    
study = optuna.create_study(direction="maximize",study_name="SAGE loss")
study.optimize(objective,n_trials = 1)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print(" Number of finid=shed trials: ",len(study.trials))
print(" Number of pruned trials: ", len(pruned_trials))
print(" Number of complete trials: ", len(complete_trials))

print('Best trial:')
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key,value))

**[5,10,25]**

In [8]:
import optuna
from torch_geometric.datasets import Planetoid
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]

def objective(trial):
    # Integer parameter
    hidden_layer = trial.suggest_categorical("hidden_layer", [32,64,128,256])
    out_layer = trial.suggest_categorical("out_layer", [32,64,128])
    # Floating point parameter
    dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
    size = [5,10,25]#list(map(lambda x: int(x),b))
    Conv ="SAGE"# trial.suggest_categorical("conv", ["SAGE","GCN","GAT"])
    
    walk_length = trial.suggest_int("walk length",5,20,step = 5)
    walks_per_node = trial.suggest_int("walks per node", 5,10,step = 5)
    context_size = trial.suggest_int("context size",5,20,step = 5)
    q = trial.suggest_float("q", 0.0,0.9,step = 0.1)
    p = trial.suggest_float("p", 0.0,0.9,step = 0.1)
    num_negative_samples = walks_per_node
    loss = {"Name": "Node2Vec","walk length":walk_length,"walks per node":walks_per_node,"num negative samples":num_negative_samples,"context size" : context_size ,"p":p ,"q":q, "loss var": "Random Walks"}#то же самое 

    model = Net(dataset = dataset,mode='unsupervised',conv=Conv,loss_function=loss["loss var"],device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = len(size),dropout = dropout)
    train_loader = NeighborSampler(data.edge_index, node_idx=train_mask, batch_size = 677, sizes=size)
    LossSampler = Sampler(data,device=device,mask=train_mask,loss_info=loss)
    
    model.to(device)
    #Generate the optimizers
#optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop","SGD"])
    learning_rate=trial.suggest_float("lr",5e-4,1e-2)
    #optimizer = getattr(optim,optimizer_name)(model.parameters(),lr = learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  
    #classifier = trial.suggest_categorical("classifier", ["logistic regression", "catboost"])
    classifier = "logistic regression"
    #classifier = "catboost"
    #training of the model
    for epoch in range(50):
        loss = train(model,data,optimizer,LossSampler,train_loader,dropout)
        train_acc, test_acc = test(model,data,classifier)
        
    trial.report(test_acc,epoch)
    if trial.should_prune():
        raise optuna.exceptions.TrialsPruned()
    return test_acc
        
    
study = optuna.create_study(direction="maximize",study_name="SAGE loss")
study.optimize(objective,n_trials = 1)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print(" Number of finid=shed trials: ",len(study.trials))
print(" Number of pruned trials: ", len(pruned_trials))
print(" Number of complete trials: ", len(complete_trials))

print('Best trial:')
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key,value))

[I 2021-04-11 15:02:21,760] A new study created in memory with name: SAGE loss
c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Plea

c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://

c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://

c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://

c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://

c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://

c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://

c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\users\anpolol\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://

Study statistics: 
 Number of finid=shed trials:  1
 Number of pruned trials:  0
 Number of complete trials:  1
Best trial:
 Value:  0.6561922365988909
 Params: 
 hidden_layer: 128
 out_layer: 128
 dropout: 0.1
 walk length: 15
 walks per node: 5
 context size: 20
 q: 0.8
 p: 0.5
 lr: 0.006811887635336468


In [9]:
import optuna
from torch_geometric.datasets import Planetoid
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]

def objective(trial):
    # Integer parameter
    hidden_layer = trial.suggest_categorical("hidden_layer", [32,64,128,256])
    out_layer = trial.suggest_categorical("out_layer", [32,64,128])
    # Floating point parameter
    dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
    size = [5,10,25]#list(map(lambda x: int(x),b))
    Conv ="SAGE"# trial.suggest_categorical("conv", ["SAGE","GCN","GAT"])
    
    walk_length = trial.suggest_int("walk length",5,20,step = 5)
    walks_per_node = trial.suggest_int("walks per node", 5,10,step = 5)
    context_size = trial.suggest_int("context size",5,20,step = 5)
    q = trial.suggest_float("q", 0.0,0.9,step = 0.1)
    p = trial.suggest_float("p", 0.0,0.9,step = 0.1)
    num_negative_samples = walks_per_node
    loss = {"Name": "Node2Vec","walk length":walk_length,"walks per node":walks_per_node,"num negative samples":num_negative_samples,"context size" : context_size ,"p":p ,"q":q, "loss var": "Random Walks"}#то же самое 

    model = Net(dataset = dataset,mode='unsupervised',conv=Conv,loss_function=loss["loss var"],device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = len(size),dropout = dropout)
    train_loader = NeighborSampler(data.edge_index, node_idx=train_mask, batch_size = 677, sizes=size)
    LossSampler = Sampler(data,device=device,mask=train_mask,loss_info=loss)
    
    model.to(device)
    #Generate the optimizers
#optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop","SGD"])
    learning_rate=trial.suggest_float("lr",5e-4,1e-2)
    #optimizer = getattr(optim,optimizer_name)(model.parameters(),lr = learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  
    #classifier = trial.suggest_categorical("classifier", ["logistic regression", "catboost"])
    #classifier = "logistic regression"
    classifier = "catboost"
    #training of the model
    for epoch in range(50):
        loss = train(model,data,optimizer,LossSampler,train_loader,dropout)
        train_acc, test_acc = test(model,data,classifier)
        
    trial.report(test_acc,epoch)
    if trial.should_prune():
        raise optuna.exceptions.TrialsPruned()
    return test_acc
        
    
study = optuna.create_study(direction="maximize",study_name="SAGE loss")
study.optimize(objective,n_trials = 1)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print(" Number of finid=shed trials: ",len(study.trials))
print(" Number of pruned trials: ", len(pruned_trials))
print(" Number of complete trials: ", len(complete_trials))

print('Best trial:')
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key,value))

[I 2021-04-11 15:33:06,156] A new study created in memory with name: SAGE loss
[I 2021-04-11 15:43:24,293] Trial 0 finished with value: 0.37707948243992606 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.0, 'walk length': 20, 'walks per node': 5, 'context size': 10, 'q': 0.7000000000000001, 'p': 0.1, 'lr': 0.009157984699291515}. Best is trial 0 with value: 0.37707948243992606.


Study statistics: 
 Number of finid=shed trials:  1
 Number of pruned trials:  0
 Number of complete trials:  1
Best trial:
 Value:  0.37707948243992606
 Params: 
 hidden_layer: 256
 out_layer: 128
 dropout: 0.0
 walk length: 20
 walks per node: 5
 context size: 10
 q: 0.7000000000000001
 p: 0.1
 lr: 0.009157984699291515
